# Neural Network

Importing the libraries


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 200)

Creating a df with the disease rates for each city and their respective RISK value (1, if it belongs to a high risk cluster, and 0 otherwise).

In [2]:
cities = pd.read_csv('../../SatScan/muncod_risk_2015_to_2017.csv', index_col=[0])

disease = ""
path = '../../TabNet/Rates/'
all_files = glob.glob(path + "*.csv")
diseases_list = []
all_diseases = cities
for file in all_files:
    file_name = file.split("\\")[1]
    disease = file_name.split("_RATE")[0]
    disease_df = pd.read_csv(path + disease + '_RATE_15_17.csv', sep=',', index_col=0)
    disease_df = disease_df[["RATE", "MUNCOD"]]
    disease_df.rename(columns={"RATE": disease}, inplace=True)

    all_diseases = pd.merge(disease_df, all_diseases, left_on="MUNCOD", right_on="MUNCOD", how="outer")
df = all_diseases.drop('MUNCOD', 1)
df = df.drop('TRAUMATISMO_INTRACRANIANO', 1) # We remove Traumatismo Intracraniano (low p-value on Spearman test)
df = df.fillna(0)

Selecting X (independent variables) and y (dependent binary variable).

In [3]:
X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

Splitting the dataset into the Training set and Test set

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Feature Scaling

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Importing the Keras libraries and packages

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Initialising the ANN

In [16]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 14))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 50, epochs = 100)

Epoch 1/100
6852/6852 [==============================] - 0s 68us/step - loss: 0.6823 - accuracy: 0.6305
Epoch 2/100
6852/6852 [==============================] - 0s 30us/step - loss: 0.5973 - accuracy: 0.7223
Epoch 3/100
6852/6852 [==============================] - 0s 26us/step - loss: 0.5475 - accuracy: 0.7239
Epoch 4/100
6852/6852 [==============================] - 0s 28us/step - loss: 0.5383 - accuracy: 0.7271
Epoch 5/100
6852/6852 [==============================] - 0s 27us/step - loss: 0.5353 - accuracy: 0.7296
Epoch 6/100
6852/6852 [==============================] - 0s 28us/step - loss: 0.5339 - accuracy: 0.7313
Epoch 7/100
6852/6852 [==============================] - 0s 27us/step - loss: 0.5334 - accuracy: 0.7312
Epoch 8/100
6852/6852 [==============================] - 0s 27us/step - loss: 0.5325 - accuracy: 0.7319
Epoch 9/100
6852/6852 [==============================] - 0s 27us/step - loss: 0.5321 - accuracy: 0.7323
Epoch 10/100
6852/6852 [==============================] - 0s 28u

6852/6852 [==============================] - 0s 30us/step - loss: 0.5172 - accuracy: 0.7462
Epoch 79/100
6852/6852 [==============================] - 0s 28us/step - loss: 0.5172 - accuracy: 0.7445
Epoch 80/100
6852/6852 [==============================] - 0s 25us/step - loss: 0.5171 - accuracy: 0.7468
Epoch 81/100
6852/6852 [==============================] - 0s 29us/step - loss: 0.5168 - accuracy: 0.7477
Epoch 82/100
6852/6852 [==============================] - 0s 27us/step - loss: 0.5167 - accuracy: 0.7461
Epoch 83/100
6852/6852 [==============================] - 0s 31us/step - loss: 0.5166 - accuracy: 0.7450
Epoch 84/100
6852/6852 [==============================] - 0s 29us/step - loss: 0.5168 - accuracy: 0.7458
Epoch 85/100
6852/6852 [==============================] - 0s 34us/step - loss: 0.5165 - accuracy: 0.7465
Epoch 86/100
6852/6852 [==============================] - 0s 34us/step - loss: 0.5165 - accuracy: 0.7462
Epoch 87/100
6852/6852 [==============================] - 0s 28us/st

Predicting the Test set results

In [17]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

Making the Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[993, 213],
       [345, 734]], dtype=int64)

In [19]:
from sklearn import metrics
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 0.7750791974656811
Recall: 0.680259499536608


# Desmembrando a média

In [2]:
cities = pd.read_csv('../../SatScan/muncod_risk_2015_to_2017.csv', index_col=[0])

df1 = pd.read_csv('../../TabNet/complete_rates_2015.csv', index_col=[0])
df2 = pd.read_csv('../../TabNet/complete_rates_2016.csv', index_col=[0])
df3 = pd.read_csv('../../TabNet/complete_rates_2017.csv', index_col=[0])
frames = [df1, df2, df3]
df = pd.concat(frames)
df = pd.merge(df, cities, left_on="MUNCOD", right_on="MUNCOD")
df = df.drop('RATE', 1)
df = df.drop('MUNCOD', 1)

## Strategy 1: don't remove anything

Selecting X (independent variables) and y (dependent binary variable).

In [3]:
X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

## Strategy 2: imput values

In [12]:
from sklearn.impute import SimpleImputer
import numpy as np

df.loc[:, df.columns != 'RISK'] = df.loc[:, df.columns != 'RISK'].replace(0, np.nan)
X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X)
X = imputer.transform(X)

Precision: 0.7570239334027055
Recall: 0.6680440771349863

## Strategy 3: remove columns + fill null
Remove columns where there are less than N non-null items
Fill null items with 0

In [126]:
import numpy as np
N = 12
df.loc[:, df.columns != 'RISK'] = df.loc[:, df.columns != 'RISK'].replace(0, np.nan)
df = df.dropna(thresh=N) # At least N non null items
df = df.fillna(0)

X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

## Strategy 4: remove columns + imput values

In [134]:
import numpy as np
from sklearn.impute import SimpleImputer
N = 12
df.loc[:, df.columns != 'RISK'] = df.loc[:, df.columns != 'RISK'].replace(0, np.nan)
df = df.dropna(thresh=N) # At least N non null items

X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X)
X = imputer.transform(X)